In [ ]:
# В этом ноутбуке векторные хранилища объединяются по двойкам, тройкам и т.д.

In [ ]:
%pip install yandex_chain 
%pip install -U langchain-community
%pip install faiss-cpu

In [8]:
import faiss
import pickle

pkl_files = [
    'faiss_2016_index.pkl',
    'faiss_2017_index.pkl',
    'faiss_2019_index.pkl',
    'faiss_2022_index.pkl',
    'faiss_2023_index.pkl'
]
index_files = [
    'faiss_2016_index.index',
    'faiss_2017_index.index',
    'faiss_2019_index.index',
    'faiss_2022_index.index',
    'faiss_2023_index.index'
] 

# Функция для загрузки индекса из pickle файла
def load_index_from_pkl(filepath):
    with open(filepath, 'rb') as f:
        index = pickle.load(f)
    return index

# Пользовательский ввод
n = int(input("Введите число от 1 до 4: "))  # 1 означает индекс, по который нужно отобрать названия хранилищ

# Отбор имен хранилищь
pkl_files_subset = pkl_files[:n+1]
print('pkl_files_subset', pkl_files_subset)
index_files_subset = index_files[:n+1]
print('index_files_subset', index_files_subset)

# Конкатенация pkl файлов
if pkl_files_subset:
    combined_pkl = load_index_from_pkl(pkl_files_subset[0])
    for pkl_file in pkl_files_subset[1:]:
        index = load_index_from_pkl(pkl_file)
        combined_pkl.merge_from(index)

    # Сохранение объединенного pkl индекса в файл
    combined_pkl_file = f'combined_{n}_index.pkl'
    with open(combined_pkl_file, 'wb') as f:
        pickle.dump(combined_pkl, f)
    print(f'Объединенный pkl индекс сохранен в файл {combined_pkl_file}')

# Конкатенация файлов с индексами
if index_files_subset:
    index1 = faiss.read_index(index_files_subset[0])
    for index_file in index_files_subset[1:]:
        index2 = faiss.read_index(index_file)
        index1.merge_from(index2)

    # Сохранение объединенного индекса в файл
    combined_index_file = f'combined_{n}_index.index'
    faiss.write_index(index1, combined_index_file)
    print(f'Объединенный индекс сохранен в файл {combined_index_file}')



Введите число от 1 до 4:  4


pkl_files_subset ['faiss_2016_index.pkl', 'faiss_2017_index.pkl', 'faiss_2019_index.pkl', 'faiss_2022_index.pkl', 'faiss_2023_index.pkl']
index_files_subset ['faiss_2016_index.index', 'faiss_2017_index.index', 'faiss_2019_index.index', 'faiss_2022_index.index', 'faiss_2023_index.index']
Объединенный pkl индекс сохранен в файл combined_4_index.pkl
Объединенный индекс сохранен в файл combined_4_index.index


In [9]:
# для наглядности, как задается ретривер 
# в зависимости от n (года, который ввел пользователь, а для программы - числа от 0 до 4)

def load_retriever(n):
    # Определение имен файлов для загрузки
    if n == 0:
        combined_index_file = 'faiss_2016_index.index'
        combined_pkl_file = 'faiss_2016_index.pkl'
    else:
        combined_index_file = f'combined_{n}_index.index'
        combined_pkl_file = f'combined_{n}_index.pkl'
    
    # Загрузка индекса из файла
    index = faiss.read_index(combined_index_file)

    # Загрузка метаданных
    with open(combined_pkl_file, "rb") as f:
        vectorstore = pickle.load(f)

    # Обновление индекса в загруженном векторном хранилище
    vectorstore.index = index

    # Создание и возврат retriever
    return vectorstore.as_retriever()

retriever = load_retriever(n)
